# 제 6 장 __지도학습의 이해__
___

## __사전설정__
---

(1) 저장소 데이터 가져오기

In [ ]:
!rm -rf /content/BizDataAnalysis/

In [ ]:
!git clone https://github.com/BizStat/BizDataAnalysis.git


(2) matplotlib 환경에서 한글 사용

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -f -v
!rm ~/.cache/matplotlib -rf

런타임 메뉴에서 '세션 다시 시작' 후 다음의 명령문 실행

In [ ]:
from matplotlib import rc
rc('font', family='NanumGothicCoding')
rc('axes', unicode_minus=False)

(3) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

___

## 6.3 __분류모형의 이해와 활용__

### (1) Default 데이터

* 데이터 가져오기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
default = pd.read_table('/content/BizDataAnalysis/DATA/Default.csv', sep=',')

In [ ]:
default

* 데이터 살펴보기

In [ ]:
default.iloc[:,[3,4]].describe()

In [ ]:
default.iloc[:,[3,4]].hist(bins=20)

In [ ]:
import seaborn as sns
sns.pairplot(default.iloc[:,1:], hue="default")
plt.show()

In [ ]:
pd.crosstab(index=default['default'],columns=default['student'])

* 데이터 분할 : 학습데이터와 평가데이터

In [ ]:
default.iloc[:,[1]]

In [ ]:
default.iloc[:,2:]

In [ ]:
X = default.iloc[:,2:]
y = default.iloc[:,[1]]

* 가변수로 변환

In [ ]:
X = pd.get_dummies(X,drop_first=True,dtype=int)

In [ ]:
y = pd.get_dummies(y,drop_first=True,dtype=int)
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=3,train_size=0.7)

In [ ]:
print(f"학습 데이터의 수 : {len(y_train)}, 학습 데이터에서 연체개체 수 : {y_train.sum()}")
print(f"평가 데이터의 수 : {len(y_test)}, 평가 데이터에서 연체개체 수 : {y_test.sum()}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=3,train_size=0.7,stratify=default['default'])

In [ ]:
print(f"학습 데이터의 수 : {len(y_train)}, 학습 데이터에서 연체개체 수 : {y_train.sum()}")
print(f"평가 데이터의 수 : {len(y_test)}, 평가 데이터에서 연체개체 수 : {y_test.sum()}")

---
__로지스틱 회귀모형__

* 로지스틱 회귀모형의 적합

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

* 로지스틱 회귀모형의 추정 결과 살펴보기

In [ ]:
for i in range(0,len(LR.feature_names_in_)+1):
  if i == 0 :
    print(f'절편 : {LR.intercept_[0]}')
  else :
    print(f'{LR.feature_names_in_[i-1]} : {LR.coef_[0,i-1]}')

In [ ]:
import numpy as np
for i in range(0,len(LR.feature_names_in_)+1):
  if i == 0 :
    print(f'절편 : {np.exp(LR.intercept_[0])}')
  else :
    print(f'{LR.feature_names_in_[i-1]} : {np.exp(LR.coef_[0,i-1])}')

* 로지스틱 회귀모형 적합 결과를 이용한 예측

In [ ]:
y_pred = LR.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred_prob = LR.predict_proba(X_test)

In [ ]:
y_pred_prob

In [ ]:
y_pred_prob[y_pred_prob[:,0]<y_pred_prob[:,1]]

In [ ]:
np.transpose(y_test).to_numpy()[0,:]

In [ ]:
y_actual = np.transpose(y_test).to_numpy()[0,:]

In [ ]:
pred_df = pd.DataFrame({
    'actual_class': y_actual,
    'predicted_class': y_pred,
    'probabilty_class_0': y_pred_prob[:,1]
})

In [ ]:
pred_df.head()

In [ ]:
pd.crosstab(index=pred_df['actual_class'],columns=pred_df['predicted_class'])

* 로지스틱 회귀모형의 성능평가

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=['default_no','default_yes']))


* 분류기준값의 변화에 따른 모형의 정확도

In [ ]:
def tmp(x):
    if x >= 0.2:
        result = 1
    else:
        result = 0
    return result

pred_df['predicted_class'] = pred_df['probabilty_class_0'].apply(tmp)
pd.crosstab(index=pred_df['actual_class'],columns=pred_df['predicted_class'])

In [ ]:
def tmp(x):
    if x >= 0.8:
        result = 1
    else:
        result = 0
    return result

pred_df['predicted_class'] = pred_df['probabilty_class_0'].apply(tmp)
pd.crosstab(index=pred_df['actual_class'],columns=pred_df['predicted_class'])

* ROC 곡선

In [ ]:
import sklearn.metrics as sm
fpr, tpr, threshold = sm.roc_curve(y_test, y_pred_prob[:, 0], pos_label=0)
sm.auc(fpr,tpr)

In [ ]:
plt.figure(figsize = (8, 8))
plt.plot(fpr, tpr, 'b')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()